In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from sklearn.preprocessing import PolynomialFeatures

from sklearn.model_selection import train_test_split
import lightgbm as lgbm
from sklearn.metrics import confusion_matrix, accuracy_score
from pycaret.classification import *

In [2]:
seed = 42
np.random.seed(seed)
set_config('seed', seed)

# 1. 데이터 로드

In [3]:
train=pd.read_csv('input/train.csv', index_col=0)#, na_values='?') #인덱스 col=0도 check!
test=pd.read_csv('input/test.csv', index_col=0)#, na_values='?')
submission=pd.read_csv('input/sample_submission.csv', index_col=0)  

In [4]:
train['income']=pd.factorize(train['income'], sort=True)[0]

In [5]:
train.shape, test.shape

((26049, 15), (6512, 14))

In [6]:
train.head(3)
test.head(3)

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
id,,,,,,,,,,,,,,,
0,40,Private,168538,HS-grad,9,Married-civ-spouse,Sales,Husband,White,Male,0,0,60,United-States,1
1,17,Private,101626,9th,5,Never-married,Machine-op-inspct,Own-child,White,Male,0,0,20,United-States,0
2,18,Private,353358,Some-college,10,Never-married,Other-service,Own-child,White,Male,0,0,16,United-States,0


,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country
id,,,,,,,,,,,,,,
0,28,Private,67661,Some-college,10,Never-married,Adm-clerical,Other-relative,White,Female,0,0,40,United-States
1,40,Self-emp-inc,37869,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,50,United-States
2,20,Private,109952,Some-college,10,Never-married,Handlers-cleaners,Own-child,White,Male,0,0,25,United-States


# 2. EDA & 전처리

In [7]:
train.info()
train.describe()
train.describe(include='O')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26049 entries, 0 to 26048
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             26049 non-null  int64 
 1   workclass       26049 non-null  object
 2   fnlwgt          26049 non-null  int64 
 3   education       26049 non-null  object
 4   education_num   26049 non-null  int64 
 5   marital_status  26049 non-null  object
 6   occupation      26049 non-null  object
 7   relationship    26049 non-null  object
 8   race            26049 non-null  object
 9   sex             26049 non-null  object
 10  capital_gain    26049 non-null  int64 
 11  capital_loss    26049 non-null  int64 
 12  hours_per_week  26049 non-null  int64 
 13  native_country  26049 non-null  object
 14  income          26049 non-null  int64 
dtypes: int64(7), object(8)
memory usage: 3.2+ MB


,age,fnlwgt,education_num,capital_gain,capital_loss,hours_per_week,income
count,26049.000000,2.604900e+04,26049.000000,26049.00000,26049.000000,26049.000000,26049.000000
mean,38.569235,1.903045e+05,10.088372,1087.68970,87.732734,40.443126,0.242044
std,13.671489,1.059663e+05,2.567610,7388.85469,403.230205,12.361850,0.428329
min,17.000000,1.376900e+04,1.000000,0.00000,0.000000,1.000000,0.000000
25%,28.000000,1.181080e+05,9.000000,0.00000,0.000000,40.000000,0.000000
50%,37.000000,1.788660e+05,10.000000,0.00000,0.000000,40.000000,0.000000
75%,48.000000,2.377350e+05,12.000000,0.00000,0.000000,45.000000,0.000000
max,90.000000,1.484705e+06,16.000000,99999.00000,4356.000000,99.000000,1.000000


,workclass,education,marital_status,occupation,relationship,race,sex,native_country
count,26049,26049,26049,26049,26049,26049,26049,26049
unique,9,16,7,15,6,5,2,41
top,Private,HS-grad,Married-civ-spouse,Prof-specialty,Husband,White,Male,United-States
freq,18123,8433,11970,3304,10541,22315,17482,23371


## Missing Value

In [8]:
train.isnull().sum()#.sum()
test.isnull().sum()#.sum()

age               0
workclass         0
fnlwgt            0
education         0
education_num     0
marital_status    0
occupation        0
relationship      0
race              0
sex               0
capital_gain      0
capital_loss      0
hours_per_week    0
native_country    0
income            0
dtype: int64

age               0
workclass         0
fnlwgt            0
education         0
education_num     0
marital_status    0
occupation        0
relationship      0
race              0
sex               0
capital_gain      0
capital_loss      0
hours_per_week    0
native_country    0
dtype: int64

### outlier

### education

In [9]:
train.drop(['education'],axis=1, inplace=True)
test.drop(['education'],axis=1, inplace=True)

### capital_gain 99999

In [10]:
df = pd.concat([train, test])
df.shape

(32561, 14)

In [11]:
df['capital_gain_imsi'] = df['capital_gain'].replace(99999,0)

In [12]:
capital_gain_imsi = pd.DataFrame(df.groupby(['education_num'])['capital_gain_imsi'].mean()).reset_index()

In [13]:
train = train.merge(capital_gain_imsi, on=['education_num'], how='left')
test = test.merge(capital_gain_imsi, on=['education_num'], how='left')

In [14]:
train.loc[train.capital_gain == 99999, 'capital_gain'] = train['capital_gain_imsi']
test.loc[test.capital_gain == 99999, 'capital_gain'] = test['capital_gain_imsi']

In [15]:
train.drop(['capital_gain_imsi'],axis=1, inplace=True)
test.drop(['capital_gain_imsi'],axis=1, inplace=True)

In [29]:
train['imsi'] = train.capital_gain*train.hours_per_week*train.fnlwgt 
test['imsi'] = test.capital_gain*test.hours_per_week*train.fnlwgt 

# 3. 모델 학습

In [17]:
import gc
gc.collect()

37

In [18]:
train.columns

Index(['age', 'workclass', 'fnlwgt', 'education_num', 'marital_status',
       'occupation', 'relationship', 'race', 'sex', 'capital_gain',
       'capital_loss', 'hours_per_week', 'native_country', 'income', 'imsi'],
      dtype='object')

In [32]:
%%time
clf = setup(session_id=seed, 
            data = train, target = 'income'
          #, categorical_features=col_cat
          , numeric_features = ['education_num']
          , ignore_features =['fnlwgt']
           )

Setup Succesfully Completed!


,Description,Value
0,session_id,42
1,Target Type,Binary
2,Label Encoded,"0: 0, 1: 1"
3,Original Data,"(26049, 15)"
4,Missing Values,False
5,Numeric Features,7
6,Categorical Features,7
7,Ordinal Features,False
8,High Cardinality Features,False
9,High Cardinality Method,None


Wall time: 3.31 s


In [33]:
best_3 = compare_models(sort = 'F1', n_select = 3
                       ,exclude=['lr', 'knn', 'nb', 'dt', 'svm', 'rbfsvm', 'ridge','qda']
                       #,exclude=['lr', 'knn', 'nb', 'dt', 'svm', 'rbfsvm', 'gpc', 'mlp', 'ridge', 'rf', 'qda', 'ada', 'xgboost']
                       )

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
0,CatBoost Classifier,0.8672,0.9241,0.6377,0.7742,0.6991,0.6150,0.6199,10.8585
1,Light Gradient Boosting Machine,0.8635,0.9213,0.6475,0.7539,0.6965,0.6091,0.6122,0.2607
2,Extreme Gradient Boosting,0.8609,0.9214,0.6429,0.7476,0.6911,0.6020,0.6050,2.0253
3,Gradient Boosting Classifier,0.8621,0.9182,0.6044,0.7766,0.6796,0.5935,0.6012,1.7269
4,Ada Boost Classifier,0.8562,0.9115,0.6125,0.7484,0.6734,0.5824,0.5874,0.7011
5,Linear Discriminant Analysis,0.8457,0.8998,0.5905,0.7220,0.6492,0.5516,0.5565,0.1776
6,Random Forest Classifier,0.8384,0.8684,0.5859,0.6988,0.6371,0.5343,0.5379,0.1263
7,Extra Trees Classifier,0.8285,0.8524,0.5976,0.6617,0.6278,0.5167,0.5181,0.5431


In [34]:
blended = blend_models(estimator_list = best_3, fold = 5, method = 'soft')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8676,0.9222,0.6424,0.7724,0.7014,0.6173,0.6216
1,0.8625,0.9163,0.6602,0.7432,0.6992,0.6105,0.6123
2,0.8543,0.9213,0.6149,0.7393,0.6714,0.5788,0.5828
3,0.8699,0.9307,0.6418,0.7811,0.7046,0.6222,0.6272
4,0.8754,0.9257,0.6715,0.7830,0.7230,0.6432,0.6464
Mean,0.8659,0.9232,0.6462,0.7638,0.6999,0.6144,0.6181
SD,0.0071,0.0048,0.0192,0.0188,0.0166,0.0209,0.0208


In [35]:
pred_holdout = predict_model(blended)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.8761,0.9293,0.6563,0.7958,0.7194,0.6408,0.6457


In [36]:
%%time
final_model = finalize_model(blended)

Wall time: 3min 1s


In [37]:
predictions = predict_model(final_model, data = test)

# 4. 예측

In [38]:
submission['prediction'] = predictions['Label']

# 5. 제출

In [39]:
submission.to_csv('output/20201023-2.csv')

In [ ]:
xgboost = create_model('xgboost')

In [ ]:
plot_model(estimator = xgboost, plot = 'feature')

In [ ]:
lightgbm = create_model('lightgbm')

In [ ]:
plot_model(estimator = lightgbm, plot = 'feature')